In [1]:
import pandas as pd 
import numpy as np
import os
import yaml
from utils.MAPPING import *
from position.flat import *

In [2]:
asset = yaml.load(open('utils/asset.yml'), Loader=yaml.FullLoader)


In [51]:
lot_multiplier = asset['cocoa']['CC_LDN']['lot_size']
lot_multiplier

10

lot_multiplier

In [4]:
avail_contracts = asset['cocoa']['CC_LDN']['chain']['futures']['month']['absolute']

In [33]:
position = pd.read_csv('data/pos_cocoa.csv')
SNAPSHOT_DATE = '24-04-2023'

{
    'commodity':  'commodity', # commodity code
    'trade_date': 'contdate', # trade date
    'trade_price': 'tprice',    # trade price
    'broker': 'broker', # broker
    'strategy': 'strategy', # strategy
    'volume': 'lots', # volume
    'currency': 'currency', # currency
    'contract': 'contract', # contract
    
}

{'commodity': 'commodity',
 'trade_date': 'contdate',
 'trade_price': 'tprice',
 'broker': 'broker',
 'strategy': 'strategy',
 'volume': 'lots',
 'currency': 'currency',
 'contract': 'contract'}

In [35]:
position


,company,market,commodity,contdate,seqno,broker,broker_name,prompt,physref,prompt_string,month,lots,tprice,broker_account,sub_account2,currency,PNL,ps,tradetype,Type Account
0,1,LNCC,CC,24/01/2023,41020,MAREX,MAREX,20230500,NaN,MAY2023,202305 (MAY2023),25,1980,08740,CC,GBP,72250,P,F,Unsecured
1,1,LNCC,CC,24/01/2023,41021,MAREX,MAREX,20230500,NaN,MAY2023,202305 (MAY2023),-47,1975,08740,CC,GBP,-138180,S,F,Unsecured
2,1,LNCC,CC,24/01/2023,41022,MAREX,MAREX,20230500,NaN,MAY2023,202305 (MAY2023),-3,1976,08740,CC,GBP,-8790,S,F,Unsecured
3,1,LNCC,CC,21/09/2022,39215,MAREX,MAREX,20230500,NaN,MAY2023,202305 (MAY2023),2,1832,08740,CC,GBP,8740,P,F,Unsecured
4,1,LNCC,CC,21/09/2022,39216,MAREX,MAREX,20230500,NaN,MAY2023,202305 (MAY2023),5,1833,08740,CC,GBP,21800,P,F,Unsecured
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287,1,NYCC,CC,17/03/2023,41946,MACQUARI,Macquarie Bank Ltd London,20230900,NaN,SEP2023,202309 (SEP2023),22,2704,NVG06,CC,USD,54780,P,F,Unsecured
1288,1,NYCC,CC,17/03/2023,41947,MACQUARI,Macquarie Bank Ltd London,20230900,NaN,SEP2023,202309 (SEP2023),1,2705,NVG06,CC,USD,2480,P,F,Unsecured
1289,1,NYCC,CC,17/03/2023,41948,MACQUARI,Macquarie Bank Ltd London,20230900,NaN,SEP2023,202309 (SEP2023),12,2706,NVG06,CC,USD,29640,P,F,Unsecured
1290,1,NYCC,CC,17/03/2023,41949,MACQUARI,Macquarie Bank Ltd London,20230900,NaN,SEP2023,202309 (SEP2023),1,2707,NVG06,CC,USD,2460,P,F,Unsecured


In [37]:
# split on numeric
cmy = position['prompt_string'].str.split('(\d+)', expand=True).rename(columns={0:'cMonth', 1:'cYear', 2:'na'}).drop(columns='na')
cmy.cMonth.replace(MMM, inplace=True)
contract = cmy.cMonth + cmy.cYear.str[-2:]
position['contract'] = contract

In [38]:
pCore = position.groupby(['market','commodity', 'contract']).agg({'lots': 'sum', 'tprice': 'mean'})
pCore.reset_index(inplace=True)
pCore = pCore.loc[pCore.lots != 0, :]

In [12]:
pCore.lots = [200,-309,-200]

portfolios = [[200,-309,-200],[150,-50,45],[100,50,600],[800,255,-200],[-1500,-250,-60]]

In [39]:


SNAPSHOT_DATE = pd.to_datetime(SNAPSHOT_DATE, format='%d-%m-%Y')
rMonth = SNAPSHOT_DATE.month
rYear = SNAPSHOT_DATE.year

avail_contracts = np.array(avail_contracts)
avail_contracts = np.sort(avail_contracts)
rel_contracts = avail_contracts[avail_contracts >= rMonth]
rC1 = rel_contracts[0]

prices = pd.read_csv('data/testprices.csv')
prices = prices[['Dates','CC1 Comdty','CC2 Comdty','CC3 Comdty']]
prices.set_index('Dates', inplace=True)
start = prices.copy()
prices = prices.pct_change()
prices.dropna(inplace=True)

prices.rename(columns={'CC1 Comdty': 'K23', 'CC2 Comdty': 'N23', 'CC3 Comdty': 'U23'}, inplace=True)
start  = start[-1:]
start.rename(columns={'CC1 Comdty': 'K23', 'CC2 Comdty': 'N23', 'CC3 Comdty': 'U23'}, inplace=True)
P_S = start.copy()

vStart = start.copy()

In [43]:
rC1

5

In [14]:
# sum portfolios
master = np.array(portfolios)
master = master.sum(axis=0)
master = master.tolist()
portfolios.insert(0,master)

In [15]:
portfolios

[[-250, -304, 185],
 [200, -309, -200],
 [150, -50, 45],
 [100, 50, 600],
 [800, 255, -200],
 [-1500, -250, -60]]

In [16]:
last_p = start.copy()

In [17]:
for port in portfolios:
    dict_conts = dict(zip(pCore.contract, port))
    print(dict_conts)

{'K23': -250, 'N23': -304, 'U23': 185}
{'K23': 200, 'N23': -309, 'U23': -200}
{'K23': 150, 'N23': -50, 'U23': 45}
{'K23': 100, 'N23': 50, 'U23': 600}
{'K23': 800, 'N23': 255, 'U23': -200}
{'K23': -1500, 'N23': -250, 'U23': -60}


In [49]:
dict_conts

{'K23': -1500, 'N23': -250, 'U23': -60}

In [50]:
prices

,K23,N23,U23
Dates,,,
04/01/2010,-0.010033,-0.009668,-0.008769
05/01/2010,0.019963,0.019219,0.018304
06/01/2010,-0.008732,-0.008381,-0.008987
07/01/2010,0.005164,0.006339,0.006953
08/01/2010,-0.003929,-0.004799,-0.003603
...,...,...,...
18/04/2023,0.016983,0.015759,0.015824
19/04/2023,0.008186,0.004047,0.003048
20/04/2023,0.002598,-0.007054,-0.006077


In [52]:
fl=flat()

pR = []
pS = []
pP = []
pF = []

for port in portfolios:
    dict_conts = dict(zip(pCore.contract, port))
    start = last_p.copy()
    start = start.apply(lambda x: x * dict_conts[x.name])
    vStart = start.copy()
    
    
    # dict_conts = dict(zip(pCore.contract, pCore.lots))

    # start = start.apply(lambda x: x * dict_conts[x.name])

    # last_px = P_S.T
    # last_px.reset_index(inplace=True)
    # last_px.columns = ['contracts','last_px']
    # last_px = dict(zip(last_px.contracts, last_px.last_px))

    # vStart = start.copy()


    # weights = [_ for _ in dict_conts.values()]
    # weights_adjuster = [1 if x > 0 else -1 for x in weights] 

    # returns = prices.copy()
    # returns = returns.multiply(weights_adjuster)

    # meanReturns = returns.mean()
    # covMatrix = returns.cov()

    # weightsA = list(np.array(weights_adjuster) * np.array(weights))

    # weightsA /= np.sum(weightsA)
    # returns['portfolio'] = returns.dot(weightsA)
    
    returns, weights, weightsA, weights_adjuster, meanReturns, covMatrix = fl.returns_weights(prices,dict_conts)
    portResults, portfolio_sims, initialPortfolio = fl.MCsimulation(10000,1,vStart,weights_adjuster,lot_multiplier,weightsA,meanReturns,covMatrix)

    pR.append(portResults)
    pS.append(portfolio_sims)
    pP.append(initialPortfolio)
    pF.append(initialPortfolio - portResults)
    
    
    print('--------------+++++++++++++++++----------------')
    print(initialPortfolio)
    print(initialPortfolio-fl.mcCVaR(portResults,0.95))
    print(initialPortfolio-fl.mcVaR(portResults,0.95))
    
    
# adj_prices = prices.apply(lambda x: x * dict_conts[x.name])
# adj_prices['sumA'] = adj_prices.sum(axis=1)

# final = adj_prices * lot_multiplier

# final = final.iloc[:-1, :]

--------------+++++++++++++++++----------------
22492170
503743.1396494694
437217.42078011483
--------------+++++++++++++++++----------------
21501470
383214.406555634
336116.4310898483
--------------+++++++++++++++++----------------
7584950
207368.80989041924
173943.6545522632
--------------+++++++++++++++++----------------
22488500
916129.0846988373
789345.7230278216
--------------+++++++++++++++++----------------
38916650
1191809.6792256832
1048517.2759447023
--------------+++++++++++++++++----------------
56805100
2550117.0610360503
2217374.999625601


In [58]:

start.apply(lambda x: x * dict_conts[x.name])


,K23,N23,U23
Dates,,,
24/04/2023,-4756500,-745750,-178260


In [59]:
dict_conts[x.name]

NameError: name 'x' is not defined

In [256]:
pReturns = pd.concat(pF, axis=1)
pReturns.columns = ['master','p1','p2','p3','p4','p5']
# pd.concat(pP, axis=1)

In [257]:
pReturns = pReturns.melt()
pReturns.columns = ['portfolio','sLoss']
# group by portfolio and add column for percentile
pReturns['percentile'] = pReturns.groupby('portfolio').sLoss.rank(pct=True)
# pReturns.groupby('portfolio').apply(lambda x:pd.qcut(x.sLoss,100, labels=False))



In [258]:
pReturns.percentile = round(pReturns.percentile * 100)

In [276]:
pReturns = pd.read_csv('data/pReturns_10day.csv')

In [277]:
pReturns

,Unnamed: 0,portfolio,sLoss,percentile
0,0,master,3.096541e+05,69.0
1,1,master,2.535177e+05,66.0
2,2,master,-6.854826e+05,12.0
3,3,master,6.592997e+05,86.0
4,4,master,-2.777735e+05,31.0
...,...,...,...,...
5999995,5999995,p5,-5.240326e+05,41.0
5999996,5999996,p5,-3.547061e+06,11.0
5999997,5999997,p5,8.885374e+05,60.0
5999998,5999998,p5,-1.913217e+06,25.0


In [278]:
aggreRet = pReturns.groupby(['portfolio','percentile']).mean('sLoss')
aggreRet.reset_index(inplace=True)

In [279]:
aggreRet

,portfolio,percentile,Unnamed: 0,sLoss
0,master,0.0,5.023803e+05,-1.733189e+06
1,master,1.0,4.998941e+05,-1.392658e+06
2,master,2.0,5.004283e+05,-1.215703e+06
3,master,3.0,5.053141e+05,-1.108065e+06
4,master,4.0,4.944598e+05,-1.028314e+06
...,...,...,...,...
601,p5,96.0,5.500160e+06,5.068387e+06
602,p5,97.0,5.500991e+06,5.425109e+06
603,p5,98.0,5.499841e+06,5.898281e+06
604,p5,99.0,5.500751e+06,6.651353e+06


In [275]:
# aggreRet.to_csv('data/aggreRet_10day.csv')

In [259]:
pReturns.to_csv('data/pReturns_1day.csv')

In [232]:
# add a column for the each percentile of the first column
pReturns['pctile'] = pd.qcut(pReturns['master'], 100, labels=False)
pReturns

,master,p1,p2,p3,p4,p5,pctile
0,309654.077668,-496176.472748,-365041.410222,-1.270503e+05,-1.809948e+05,-2.280976e+06,68
1,253517.675573,385967.819669,-278417.445667,-7.246439e+05,-8.623836e+05,4.782091e+06,65
2,-685482.633129,288832.340494,252043.904206,3.789772e+05,-3.710041e+05,-2.577850e+06,11
3,659299.727540,23197.079712,261783.404489,2.702494e+05,2.348352e+06,1.026459e+06,86
4,-277773.504290,-355355.189296,109733.966713,4.428156e+05,-1.063301e+05,-1.725553e+05,30
...,...,...,...,...,...,...,...
999995,-533515.627657,638858.355983,-416018.929199,6.509985e+05,1.868821e+06,-5.240326e+05,17
999996,-385849.638503,665780.133097,-54072.964381,5.443535e+05,6.705633e+04,-3.547061e+06,24
999997,-168792.852901,-32589.015561,-72400.542773,-2.154873e+06,1.442133e+06,8.885374e+05,37
999998,277484.361586,-6109.827238,-299042.494538,1.512302e+05,-1.553816e+05,-1.913217e+06,67


In [237]:
pReturns.sort_values(by = all , inplace=True)

KeyError: <built-in function all>

In [235]:
pReturns

,master,p1,p2,p3,p4,p5,pctile
456709,-2.831511e+06,-2.988249e+05,-198914.587673,-4.676010e+04,7.295615e+05,5.660812e+05,0
718384,-2.788880e+06,-2.738066e+03,29578.262743,-8.012128e+05,2.755328e+06,-2.295399e+06,0
709625,-2.739192e+06,-9.830837e+05,85167.828537,2.437042e+05,-2.114305e+06,-3.258810e+06,0
422889,-2.738301e+06,-1.222279e+06,-424602.579895,-4.164915e+05,1.906712e+06,-3.519060e+06,0
285897,-2.729488e+06,2.494749e+05,-106832.215801,-2.803188e+05,2.338047e+06,-1.212137e+06,0
...,...,...,...,...,...,...,...
279498,2.501808e+06,6.342362e+04,-185473.693261,1.056378e+06,-1.155569e+06,-2.281563e+05,99
789786,2.504284e+06,-3.763620e+05,-642261.377773,-7.046139e+05,-8.530758e+05,1.157285e+06,99
252339,2.512503e+06,-2.000419e+04,-17127.612676,1.913127e+06,1.058175e+06,1.585996e+06,99
47659,2.785253e+06,4.734701e+05,-80509.074029,-9.900217e+05,3.166904e+05,3.707284e+06,99


In [233]:
pReturns.to_csv('data/pReturns_10day.csv')

In [198]:
dict_conts = dict(zip(pCore.contract, portfolios[5]))
start = last_p.copy()
start = start.apply(lambda x: x * dict_conts[x.name])
vStart = start.copy()

In [199]:
dict_conts = dict(zip(pCore.contract, port))
returns, weights, weightsA, weights_adjuster, meanReturns, covMatrix = fl.returns_weights(prices,dict_conts)

In [200]:
portResults, portfolio_sims, initialPortfolio = fl.MCsimulation(10000,1,vStart,weights_adjuster,lot_multiplier,weightsA,meanReturns,covMatrix)

In [203]:
initialPortfolio-fl.mcCVaR(portResults,0.95)

1185749.8805099055

In [186]:
((vStart.multiply(weights_adjuster)*lot_multiplier).sum(axis=1)).values[0]

-91250

In [184]:
pP[0] - fl.mcCVaR(pR[0],0.95)

702.0639643152499

In [167]:

portResults = pd.Series(pS[5][-1,:])
VaR = initialPortfolio - fl.mcVaR(portResults, alpha=5)
CVaR = initialPortfolio - fl.mcCVaR(portResults, alpha=5)

print('VaR ${}'.format(round(VaR,2)))
print('CVaR ${}'.format(round(CVaR,2)))

VaR $2444.21
CVaR $3068.37


In [127]:
weights_adjuster

[-1, -1, -1]

In [133]:
vStart

,K23,N23,U23
Dates,,,
24/04/2023,1141560000000000000,45393159290625000,38504160000000000


In [11]:
dict_conts

{'K23': 200, 'N23': -309, 'U23': -200}

In [13]:
NUM[rMonth] + str(rYear)[-2:]

'J23'

In [14]:
avail_contracts

[3, 5, 7, 9, 12, 15, 17, 19, 21, 24, 27, 29, 31, 33]

In [15]:
# find nearest value to rMonth in avail_contracts round up
avail_contracts = np.array(avail_contracts)
avail_contracts = np.sort(avail_contracts)
rel_contracts = avail_contracts[avail_contracts >= rMonth]
rC1 = rel_contracts[0]
# if value in array greater than 12 subtract 12


In [16]:
np.vectorize(NUM.get)(rel_contracts)
# .apply(lambda x: NUM[x])

array(['K', 'N', 'U', 'Z', 'None', 'None', 'None', 'None', 'None', 'None',
       'None', 'None', 'None'], dtype='<U4')

In [17]:
prices = pd.read_csv('data/testprices.csv')
prices = prices[['Dates','CC1 Comdty','CC2 Comdty','CC3 Comdty']]
prices.set_index('Dates', inplace=True)
start = prices.copy()
# prices.reset_index(inplace=True)

In [18]:
# log returns of dataframe
# prices = prices.apply(np.log).diff()
prices = prices.pct_change()
# prices = prices.diff()
prices.dropna(inplace=True)



In [19]:
prices

,CC1 Comdty,CC2 Comdty,CC3 Comdty
Dates,,,
04/01/2010,-0.010033,-0.009668,-0.008769
05/01/2010,0.019963,0.019219,0.018304
06/01/2010,-0.008732,-0.008381,-0.008987
07/01/2010,0.005164,0.006339,0.006953
08/01/2010,-0.003929,-0.004799,-0.003603
...,...,...,...
18/04/2023,0.016983,0.015759,0.015824
19/04/2023,0.008186,0.004047,0.003048
20/04/2023,0.002598,-0.007054,-0.006077


In [20]:
lot_multiplier

10

In [21]:
prices.rename(columns={'CC1 Comdty': 'K23', 'CC2 Comdty': 'N23', 'CC3 Comdty': 'U23'}, inplace=True)
start  = start[-1:]
start.rename(columns={'CC1 Comdty': 'K23', 'CC2 Comdty': 'N23', 'CC3 Comdty': 'U23'}, inplace=True)
P_S = start.copy()
start = start.apply(lambda x: x * dict_conts[x.name])

In [22]:
last_px = P_S.T
last_px.reset_index(inplace=True)
last_px.columns = ['contracts','last_px']
last_px = dict(zip(last_px.contracts, last_px.last_px))

In [23]:
adj_prices = prices.apply(lambda x: x * dict_conts[x.name])
adj_prices['sumA'] = adj_prices.sum(axis=1)

In [24]:
final = adj_prices * lot_multiplier

In [25]:
# remove last row 
final = final.iloc[:-1, :]

In [26]:
final

,K23,N23,U23,sumA
Dates,,,,
04/01/2010,-20.066890,29.873112,17.538555,27.344777
05/01/2010,39.926290,-59.386821,-36.607688,-56.068219
06/01/2010,-17.464619,25.896438,17.974835,26.406654
07/01/2010,10.328068,-19.586478,-13.905683,-23.164093
08/01/2010,-7.857359,14.829034,7.205044,14.176719
...,...,...,...,...
17/04/2023,12.734584,-23.465654,-15.251300,-25.982370
18/04/2023,33.966034,-48.694758,-31.647747,-46.376471
19/04/2023,16.371971,-12.505902,-6.095496,-2.229427


In [27]:
def mcVaR(returns,alpha):
    return np.percentile(returns,alpha)

def mcCVaR(returns,alpha):
    belowVar = returns[returns <= mcVaR(returns,alpha)]
    return np.mean(belowVar)



In [28]:
# np.mean(final.sumA)

meanReturns = final.sumA.mean()
covMatrix = final[final.columns.difference(['sumA'])].cov()


In [29]:
meanReturns, covMatrix

(-0.16209454295491335,
              K23          N23          U23
 K23  1124.712474 -1505.917721  -922.270829
 N23 -1505.917721  2246.120075  1378.057283
 U23  -922.270829  1378.057283   852.543271)

In [450]:
T = 1
weights = np.array([1])
meanM = np.full(shape=(T,len(weights)), fill_value=meanReturns)

mc_sims = 400 # number of simulations
T = 100 #timeframe in days
meanM = np.full(shape=(T, len(weights)), fill_value=meanReturns)
meanM = meanM.T
portfolio_sims = np.full(shape=(T, mc_sims), fill_value=0.0)


ValueError: could not broadcast input array from shape (3,) into shape (1,1)

In [396]:
np.random.normal(size=(T, len(weights)))


array([[-1.8442135 ],
       [-0.86050268],
       [ 0.70252281],
       [ 0.47491825],
       [ 0.50952667],
       [ 1.09959971],
       [ 0.38754621],
       [-0.34052418],
       [-1.32670212],
       [-0.24682495],
       [ 0.3069356 ],
       [ 0.75711885],
       [ 1.02529677],
       [ 0.59900126],
       [-0.58128333],
       [ 1.0964558 ],
       [ 0.95265949],
       [-1.08735421],
       [-1.11898285],
       [-0.39375862],
       [-0.20238025],
       [-0.76745113],
       [-0.76798861],
       [-0.93348253],
       [-0.17726321],
       [ 0.85681566],
       [ 0.180848  ],
       [ 0.67209051],
       [ 0.19001085],
       [-0.9193543 ],
       [-0.63970352],
       [ 0.79756335],
       [-0.43988804],
       [ 1.59057154],
       [ 1.29040951],
       [ 0.90563791],
       [-0.58076386],
       [ 0.53722486],
       [ 0.27011562],
       [ 0.18718545],
       [ 0.91027719],
       [-0.37606039],
       [-0.67728446],
       [-0.41318668],
       [ 1.74128495],
       [ 0

In [397]:
for m in range(0, mc_sims):
    # MC loops
    Z = np.random.normal(size=(T, len(weights)))
    # L = np.linalg.cholesky(covMatrix)
    # dailyReturns = meanM + np.inner(L, Z)
    dailyReturns = meanM + Z
    portfolio_sims[:,m] = np.cumprod(np.inner(weights, dailyReturns.T)+1)

ValueError: shapes (1,) and (100,100) not aligned: 1 (dim 0) != 100 (dim 0)

In [117]:
mc_sims = 400 # number of simulations
T = 100 #timeframe in days
meanM = np.full(shape=(T, len(weights)), fill_value=meanReturns)
meanM = meanM.T

NameError: name 'meanReturns' is not defined

In [ ]:
# montecarlo simulation Value at Risk
# 1. calculate log returns
# 2. calculate mean and standard deviation of log returns
# 3. calculate drift
# 4. calculate standard deviation of drift
# 5. calculate Z
# 6. calculate daily returns
# 7. calculate price
# 8. calculate VaR




In [254]:
prices = prices.iloc[:-1, :]

In [253]:
prices

,K23,N23,U23
Dates,,,
04/01/2010,-0.010033,-0.009668,-0.008769
05/01/2010,0.019963,0.019219,0.018304
06/01/2010,-0.008732,-0.008381,-0.008987
07/01/2010,0.005164,0.006339,0.006953
08/01/2010,-0.003929,-0.004799,-0.003603
...,...,...,...
18/04/2023,0.016983,0.015759,0.015824
19/04/2023,0.008186,0.004047,0.003048
20/04/2023,0.002598,-0.007054,-0.006077


In [321]:
prices.drop(columns='portfolio', inplace=True)

C:\Users\tmeyl\AppData\Local\Temp\ipykernel_1708\490494678.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices.drop(columns='portfolio', inplace=True)


In [451]:
prices

,K23,N23,U23
Dates,,,
04/01/2010,-0.010033,-0.009668,-0.008769
05/01/2010,0.019963,0.019219,0.018304
06/01/2010,-0.008732,-0.008381,-0.008987
07/01/2010,0.005164,0.006339,0.006953
08/01/2010,-0.003929,-0.004799,-0.003603
...,...,...,...
18/04/2023,0.016983,0.015759,0.015824
19/04/2023,0.008186,0.004047,0.003048
20/04/2023,0.002598,-0.007054,-0.006077


In [310]:


# weights = np.array(weights)

In [311]:
weights

[-100, -309, -200]

C:\Users\tmeyl\AppData\Local\Temp\ipykernel_1708\638576346.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returns['portfolio'] = returns.dot(weightsA)


[-1, -1, -1]

[100, 309, 200]

In [452]:

weights = [_ for _ in dict_conts.values()]
weights_adjuster = [1 if x > 0 else -1 for x in weights] 

returns = prices.copy()
returns = returns.multiply(weights_adjuster)

meanReturns = returns.mean()
covMatrix = returns.cov()

weightsA = list(np.array(weights_adjuster) * np.array(weights))

weightsA /= np.sum(weightsA)
returns['portfolio'] = returns.dot(weightsA)

In [484]:
vStart = start.copy()


In [485]:
mc_sims = 4000 # number of simulations
T = 1 #timeframe in days
meanM = np.full(shape=(T, len(weightsA)), fill_value=meanReturns)
meanM = meanM.T
portfolio_sims = np.full(shape=(T, mc_sims), fill_value=0.0)
initialPortfolio = ((vStart.multiply(weights_adjuster)*lot_multiplier).sum(axis=1)).values[0]
for m in range(0, mc_sims):
    # MC loops
    Z = np.random.normal(size=(T, len(weightsA)))
    L = np.linalg.cholesky(covMatrix)
    dailyReturns = meanM + np.inner(L, Z)
    # portfolio_sims[:,m] = np.cumprod(np.inner(weights, dailyReturns.T)+1)
    portfolio_sims[:,m] = np.cumprod(np.inner(weightsA, dailyReturns.T)+1)*initialPortfolio

In [486]:
portResults = pd.Series(portfolio_sims[-1,:])
VaR = initialPortfolio - mcVaR(portResults, alpha=5)
CVaR = initialPortfolio - mcCVaR(portResults, alpha=5)

print('VaR ${}'.format(round(VaR,2)))
print('CVaR ${}'.format(round(CVaR,2)))

VaR $231427.35
CVaR $289250.68


In [474]:
vStart

21501470

In [465]:
((start.multiply(weights_adjuster)*lot_multiplier).sum(axis=1)).values[0]

-88174700

In [500]:
fl=flat()
returns, weights, weightsA, weights_adjuster, meanReturns, covMatrix = fl.returns_weights(prices,dict_conts)
portResults, portfolio_sims, initialPortfolio = fl.MCsimulation(1000000,1,vStart,weights_adjuster,lot_multiplier,weightsA,meanReturns,covMatrix)

In [501]:
portResults = pd.Series(portfolio_sims[-1,:])
VaR = initialPortfolio - mcVaR(portResults, alpha=5)
CVaR = initialPortfolio - mcCVaR(portResults, alpha=5)

print('VaR ${}'.format(round(VaR,2)))
print('CVaR ${}'.format(round(CVaR,2)))

VaR $230151.27
CVaR $287962.32


In [505]:
weights

[200, -309, -200]

In [504]:
dict_conts

{'K23': 200, 'N23': -309, 'U23': -200}

In [21]:
position

,company,market,commodity,contdate,seqno,broker,broker_name,prompt,physref,prompt_string,month,lots,tprice,series,broker_account,sub_account2,currency,PNL,ps,tradetype
0,1,LNCC,CC,15/12/2022,40688,MAREX,MAREX,20230500,NaN,MAY2023,202305 (MAY2023),-200,50,2000,08740,CC,GBP,-224000,S,C
1,1,LNCC,CC,15/12/2022,40687,MAREX,MAREX,20230500,NaN,MAY2023,202305 (MAY2023),-300,51,2000,08740,CC,GBP,-333000,S,C
2,1,LNCC,CC,14/11/2022,40012,MAREX,MAREX,20230500,NaN,MAY2023,202305 (MAY2023),250,81,2000,08740,CC,GBP,202500,P,C
3,1,LNCC,CC,14/11/2022,40011,MAREX,MAREX,20230500,NaN,MAY2023,202305 (MAY2023),150,81,2000,08740,CC,GBP,121500,P,C
4,1,LNCC,CC,14/11/2022,40010,MAREX,MAREX,20230500,NaN,MAY2023,202305 (MAY2023),100,79,2000,08740,CC,GBP,83000,P,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,1,LNCC,CC,44841,39477,STONEX,StoneX,20230500,NaN,MAY2023,202305 (MAY2023),-50,81,2000,LCF10019,CC,GBP,-40500,S,C
111,1,LNCC,CC,44841,39476,STONEX,StoneX,20230500,NaN,MAY2023,202305 (MAY2023),-50,75,2000,LCF10019,CC,GBP,-43500,S,C
112,1,LNCC,CC,44841,39475,STONEX,StoneX,20230500,NaN,MAY2023,202305 (MAY2023),-50,75,2000,LCF10019,CC,GBP,-43500,S,C
113,1,LNCC,CC,44838,39397,STONEX,StoneX,20230500,NaN,MAY2023,202305 (MAY2023),200,69,2000,LCF10019,CC,GBP,186000,P,C


In [20]:
position = pd.read_csv('data/optionstest.csv')

In [22]:
cmy = position['prompt_string'].str.split('(\d+)', expand=True).rename(columns={0:'cMonth', 1:'cYear', 2:'na'}).drop(columns='na')
cmy.cMonth.replace(MMM, inplace=True)
contract = cmy.cMonth + cmy.cYear.str[-2:]
position['contract'] = contract

pCore = position.groupby(['market','commodity', 'contract','tradetype', 'series']).agg({'lots': 'sum', 'tprice': 'mean'})
pCore.reset_index(inplace=True)
pCore = pCore.loc[pCore.lots != 0, :]

In [23]:
position.columns

Index(['company', 'market', 'commodity', 'contdate', 'seqno', 'broker',
       'broker_name', 'prompt', 'physref', 'prompt_string', 'month', 'lots',
       'tprice', 'series', 'broker_account', 'sub_account2', 'currency', 'PNL',
       'ps', 'tradetype', 'contract'],
      dtype='object')

In [24]:
pCore

,market,commodity,contract,tradetype,series,lots,tprice
3,LNCC,CC,N23,C,2300,600,39.4


In [25]:
position

,company,market,commodity,contdate,seqno,broker,broker_name,prompt,physref,prompt_string,...,lots,tprice,series,broker_account,sub_account2,currency,PNL,ps,tradetype,contract
0,1,LNCC,CC,15/12/2022,40688,MAREX,MAREX,20230500,NaN,MAY2023,...,-200,50,2000,08740,CC,GBP,-224000,S,C,K23
1,1,LNCC,CC,15/12/2022,40687,MAREX,MAREX,20230500,NaN,MAY2023,...,-300,51,2000,08740,CC,GBP,-333000,S,C,K23
2,1,LNCC,CC,14/11/2022,40012,MAREX,MAREX,20230500,NaN,MAY2023,...,250,81,2000,08740,CC,GBP,202500,P,C,K23
3,1,LNCC,CC,14/11/2022,40011,MAREX,MAREX,20230500,NaN,MAY2023,...,150,81,2000,08740,CC,GBP,121500,P,C,K23
4,1,LNCC,CC,14/11/2022,40010,MAREX,MAREX,20230500,NaN,MAY2023,...,100,79,2000,08740,CC,GBP,83000,P,C,K23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,1,LNCC,CC,44841,39477,STONEX,StoneX,20230500,NaN,MAY2023,...,-50,81,2000,LCF10019,CC,GBP,-40500,S,C,K23
111,1,LNCC,CC,44841,39476,STONEX,StoneX,20230500,NaN,MAY2023,...,-50,75,2000,LCF10019,CC,GBP,-43500,S,C,K23
112,1,LNCC,CC,44841,39475,STONEX,StoneX,20230500,NaN,MAY2023,...,-50,75,2000,LCF10019,CC,GBP,-43500,S,C,K23
113,1,LNCC,CC,44838,39397,STONEX,StoneX,20230500,NaN,MAY2023,...,200,69,2000,LCF10019,CC,GBP,186000,P,C,K23


In [26]:
import torch
from datetime import datetime

expiry_date = '2023-08-04'
expiry_date = datetime.strptime(expiry_date, '%Y-%m-%d')


In [28]:
current_date = start.index[0]
current_date = datetime.strptime(current_date, '%d/%m/%Y')

(expiry_date - current_date).days/365

0.27945205479452057

In [29]:
current_date

datetime.datetime(2023, 4, 24, 0, 0)

In [283]:
import torch
def blackScholes_pyTorch(S_0, strike, time_to_expiry, implied_vol, riskfree_rate):
    S = S_0
    K = strike
    dt = time_to_expiry
    sigma = implied_vol
    r = riskfree_rate
    Phi = torch.distributions.Normal(0,1).cdf
    d_1 = (torch.log(S_0 / K) + (r+sigma**2/2)*dt) / (sigma*torch.sqrt(dt))
    d_2 = d_1 - sigma*torch.sqrt(dt)
    return S*Phi(d_1) - K*torch.exp(-r*dt)*Phi(d_2)



In [284]:
start

,K23,N23,U23
Dates,,,
24/04/2023,-4756500,-745750,-178260


In [285]:
npv_pytorch.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [286]:
from scipy.optimize import bisect

In [287]:
DEFAULT_IV_SEARCH_RANGE = (0.001, 100.0)
def err(vol):
    sigma = vol
    option = blackScholes_pyTorch(S_0, K, T, sigma, r)
    return option.item() - 90

bisect(err, DEFAULT_IV_SEARCH_RANGE[0], DEFAULT_IV_SEARCH_RANGE[1], xtol=1e-5)


0.0893689626455307

In [288]:
# black-scholes inputs



stock = torch.tensor(2983.0, requires_grad=True, device='cuda:0')
strike = torch.tensor(3000.0, requires_grad=True, device='cuda:0')
rate = torch.tensor(0.035, requires_grad=True, device='cuda:0')
vol = torch.tensor(0.2385, requires_grad=True, device='cuda:0')
time = torch.tensor((expiry_date - current_date).days/365, requires_grad=True, device='cuda:0')
dividend = torch.tensor(0.0, requires_grad=True, device='cuda:0')

# utility functions
cdf = torch.distributions.Normal(0,1).cdf
pdf = lambda x: torch.distributions.Normal(0,1).log_prob(x).exp()

In [289]:
%%time

# geometric brownian motion (single time step)
torch.manual_seed(42)
scenarios = 1000000
dW = vol * time**0.5 * torch.randn(size=(scenarios,), device='cuda:0')
r = torch.exp((rate - dividend - vol*vol/2) * time + dW)

# put option payoff & discounting
payoff = torch.max(strike - stock*r, torch.zeros(size=(scenarios,), device='cuda:0'))
ov = torch.mean(payoff) * torch.exp(-rate*time)

# run backpropagation to calculate option greeks
ov.backward()
delta = stock.grad
rho = rate.grad
vega = vol.grad
theta = time.grad
epsilon = dividend.grad
strike_greek = strike.grad

CPU times: total: 1.3 s
Wall time: 1.42 s


In [290]:
delta

tensor(-0.4627, device='cuda:0')

In [61]:
    def shock_map(last_px, returns, window,bumps = [-2,-1.5,-1,-0.5,0,0.5,1,1.5,2]):
        shock_map = returns.copy()
        shock_map = shock_map[-window:].std()
        shock_map = shock_map.to_frame()
        shock_map.reset_index(inplace=True)
        shock_map = shock_map.rename(columns={'index': 'contract', 0: 'sdev'})
        dL = []
        for i, p in enumerate(shock_map.sdev):
            dL.append(dict(zip(bumps, [p*x for x in bumps])))
            
        dL = pd.DataFrame(dL)
        dL.index = shock_map.contract
        dL.reset_index(inplace=True)
        
        dL = dL.T
        dL.columns = dL.iloc[0]
        dL = dL.iloc[1:,:]
        dL= dL.apply(lambda x: (1+x) * last_px[x.name], axis=0)
                
        
        # for k,v in last_px.items():
        #     dL[dL.select_dtypes("number").columns].loc[:,dL.contract == k] *= v 
        #     dL[dL.select_dtypes("number").columns].loc[:,dL.contract == k] += v


        return shock_map, dL
    

In [62]:
last_px

NameError: name 'last_px' is not defined

In [32]:
s, l = shock_map(last_px,prices, 30)
t = l.T
t.reset_index(inplace=True)
# create a dictionary of contracts and prices from t
t = dict(zip(t.contract, t.iloc[:,1:].values.tolist()))



NameError: name 'shock_map' is not defined

In [293]:
expiry_dict = {'N23':0.10684931506849316 , 'U23': 0.27945205479452057}

In [294]:
%%time
S_0 = torch.tensor([2983.], requires_grad=True)
K = torch.tensor([3000.], requires_grad=True)
T = torch.tensor([0.10684931506849316], requires_grad=True)
sigma = torch.tensor([0.3], requires_grad=True)
r = torch.tensor([0.035], requires_grad=True)

npv_pytorch = blackScholes_pyTorch(S_0, K, T, sigma, r)


CPU times: total: 0 ns
Wall time: 967 µs


In [295]:
t

{'K23': [3090.6870557078423,
  3110.7652917808814,
  3130.843527853921,
  3150.9217639269605,
  3171.0,
  3191.0782360730395,
  3211.1564721460786,
  3231.234708219118,
  3251.312944292158],
 'N23': [2919.2305934713636,
  2935.1729451035226,
  2951.1152967356816,
  2967.0576483678406,
  2983.0,
  2998.942351632159,
  3014.8847032643184,
  3030.8270548964774,
  3046.7694065286364],
 'U23': [2913.502027575457,
  2927.8765206815924,
  2942.2510137877284,
  2956.625506893864,
  2971.0,
  2985.3744931061356,
  2999.748986212272,
  3014.1234793184076,
  3028.497972424543]}

In [350]:
options_greeks = []

for c,v in expiry_dict.items():
    mapOp = []
    for K in [3000.,3050.,3100.,3150.,4000.]:
        res = []
        for S in t[c]:
            print(c,K,v)
            S_0 = torch.tensor([S], requires_grad=True)
            K = torch.tensor([K], requires_grad=True)
            T = torch.tensor([v], requires_grad=True)
            sigma = torch.tensor([0.3], requires_grad=True)
            r = torch.tensor([0.035], requires_grad=True)

            npv_pytorch = blackScholes_pyTorch(S_0, K, T, sigma, r)
            # npv_pytorch.backward()
            
            gradient = torch.autograd.grad(npv_pytorch, S_0, create_graph=True)
            delta, =  gradient
            delta.backward(retain_graph=True)
        
            results = {
                'S_0': S,
                'strike': K.item(),
                'price': npv_pytorch.item(),
                'rho': r.grad.item(), 
                'vega': sigma.grad.item(), 
                'theta': T.grad.item(), 
                # 'epsilon': 0.0, 
                'strike_greek': K.grad.item(),
                'delta': delta.item(),
                'gamma': S_0.grad.item(),
                
            }
            
            res.append(results)
        res = pd.DataFrame(res)
        mapOp.append(res)
        
    options_greeks.append({c: mapOp})
        

N23 3000.0 0.10684931506849316
N23 tensor([3000.], requires_grad=True) 0.10684931506849316
N23 tensor([3000.], requires_grad=True) 0.10684931506849316
N23 tensor([3000.], requires_grad=True) 0.10684931506849316
N23 tensor([3000.], requires_grad=True) 0.10684931506849316
N23 tensor([3000.], requires_grad=True) 0.10684931506849316
N23 tensor([3000.], requires_grad=True) 0.10684931506849316
N23 tensor([3000.], requires_grad=True) 0.10684931506849316
N23 tensor([3000.], requires_grad=True) 0.10684931506849316
N23 3050.0 0.10684931506849316
N23 tensor([3050.], requires_grad=True) 0.10684931506849316
N23 tensor([3050.], requires_grad=True) 0.10684931506849316
N23 tensor([3050.], requires_grad=True) 0.10684931506849316
N23 tensor([3050.], requires_grad=True) 0.10684931506849316
N23 tensor([3050.], requires_grad=True) 0.10684931506849316
N23 tensor([3050.], requires_grad=True) 0.10684931506849316
N23 tensor([3050.], requires_grad=True) 0.10684931506849316
N23 tensor([3050.], requires_grad=True

In [351]:
options_greeks

[{'N23': [           S_0  strike       price       rho      vega     theta  \
   0  2919.230593  3000.0   84.047241  0.426816  0.377628  0.669942   
   1  2935.172945  3000.0   90.984131  0.430707  0.307890  0.573315   
   2  2951.115297  3000.0   98.270142  0.433284  0.236515  0.473958   
   3  2967.057648  3000.0  105.905273  0.434544  0.164168  0.372808   
   4  2983.000000  3000.0  113.888672  0.434500  0.091513  0.270797   
   5  2998.942352  3000.0  122.218384  0.433170  0.019204  0.168851   
   6  3014.884703  3000.0  130.891602  0.430587 -0.052132  0.067859   
   7  3030.827055  3000.0  139.904663  0.426794 -0.121893 -0.031317   
   8  3046.769407  3000.0  149.252197  0.421841 -0.189515 -0.127870   
   
      strike_greek     delta     gamma  
   0     -0.001332  0.424206  0.001368  
   1     -0.001344  0.446067  0.001373  
   2     -0.001352  0.467972  0.001374  
   3     -0.001356  0.489856  0.001371  
   4     -0.001355  0.511654  0.001363  
   5     -0.001351  0.533301  0.0

In [354]:
options_greeks[0]['N23']

,S_0,strike,price,rho,vega,theta,strike_greek,delta,gamma
0,2919.230593,4000.0,0.068120,0.003295,0.032490,0.046691,-0.000008,0.000890,0.000011
1,2935.172945,4000.0,0.083754,0.003914,0.037923,0.054520,-0.000009,0.001073,0.000012
2,2951.115297,4000.0,0.102604,0.004630,0.044078,0.063395,-0.000011,0.001289,0.000015
3,2967.057648,4000.0,0.125150,0.005455,0.051022,0.073414,-0.000013,0.001543,0.000017
4,2983.000000,4000.0,0.151974,0.006403,0.058815,0.084664,-0.000015,0.001840,0.000020
5,2998.942352,4000.0,0.184068,0.007487,0.067526,0.097249,-0.000018,0.002185,0.000023
6,3014.884703,4000.0,0.221930,0.008722,0.077220,0.111262,-0.000020,0.002587,0.000027
7,3030.827055,4000.0,0.266891,0.010123,0.087955,0.126792,-0.000024,0.003051,0.000031
8,3046.769407,4000.0,0.319681,0.011705,0.099792,0.143927,-0.000027,0.003586,0.000036


In [375]:

dfs = []
for i in options_greeks:
    for k,v in i.items():
        for j in v:
        
            df = j
            df.index = [-2,-1.5,-1,-0.5,0,0.5,1,1.5,2]
            df['contract'] = k
            df.reset_index(inplace=True)
            df = df.rename(columns={'index': 'bump'})
            dfs.append(df)
#     pd.DataFrame(options_greeks[0]['N23'])
# [-2,-1.5,-1,-0.5,0,0.5,1,1.5,2]

dfs = pd.concat(dfs)
dfs


,level_0,bump,S_0,strike,price,rho,vega,theta,strike_greek,delta,gamma,contract
0,-2.0,-2.0,2919.230593,3000.0,84.047241,0.426816,0.377628,0.669942,-0.001332,0.424206,0.001368,N23
1,-1.5,-1.5,2935.172945,3000.0,90.984131,0.430707,0.307890,0.573315,-0.001344,0.446067,0.001373,N23
2,-1.0,-1.0,2951.115297,3000.0,98.270142,0.433284,0.236515,0.473958,-0.001352,0.467972,0.001374,N23
3,-0.5,-0.5,2967.057648,3000.0,105.905273,0.434544,0.164168,0.372808,-0.001356,0.489856,0.001371,N23
4,0.0,0.0,2983.000000,3000.0,113.888672,0.434500,0.091513,0.270797,-0.001355,0.511654,0.001363,N23
...,...,...,...,...,...,...,...,...,...,...,...,...
4,0.0,0.0,2971.000000,4000.0,7.487022,0.156250,0.559485,0.319881,-0.000140,0.041434,0.000188,U23
5,0.5,0.5,2985.374493,4000.0,8.102280,0.164642,0.580055,0.331974,-0.000147,0.044205,0.000197,U23
6,1.0,1.0,2999.748986,4000.0,8.758453,0.173283,0.600569,0.344067,-0.000155,0.047108,0.000207,U23
7,1.5,1.5,3014.123479,4000.0,9.457291,0.182166,0.620969,0.356130,-0.000163,0.050148,0.000216,U23


In [360]:
dfs.to_csv('data/option_greeks.csv')

In [376]:
position_options = {
    's1': {
        'N23': {3000.0: 250, 3050.0: 50, 3100.0: -250, 3150.0: -500, 4000.0: 250},
        'U23': {3000.0: 250, 3050.0: 50, 3100.0: -250, 3150.0: -500, 4000.0: 250}
        },
    's2': {
        'N23': {3000.0: -2000, 3050.0: 500, 3100.0: 250, 3150.0: -500, 4000.0: 250},
        'U23': {3000.0: 750, 3050.0: -650, 3100.0: -250, 3150.0: -500, 4000.0: 1000}
        },
}

In [379]:
strats =[]
for k,v in position_options.items():
    
    df = dfs.copy()
    df['position'] = df.apply(lambda x: v[x.contract][x.strike], axis=1)
    cols = ['rho','vega','theta','strike_greek','delta','gamma']
    df[cols] = df[cols].multiply(df.position, axis=0)
    df['strategy'] = k
    strats.append(df)
    

In [381]:
dfs = pd.concat(strats)
dfs.to_csv('data/option_greeks.csv')

In [366]:
dfs.groupby(['contract','bump','strike']).sum()

S_0       price       rho      vega     theta  \
contract bump strike                                                          
N23      -2.0 3000.0  2919.230593   84.047241  0.426816  0.377628  0.669942   
              3050.0  2919.230593   66.389832  0.407454  0.570606  0.934510   
              3100.0  2919.230593   51.719666  0.378622  0.722290  1.138008   
              3150.0  2919.230593   39.737976  0.342915  0.825342  1.270979   
              4000.0  2919.230593    0.068120  0.003295  0.032490  0.046691   
...                           ...         ...       ...       ...       ...   
U23       2.0 3000.0  3028.497972  219.822144  0.688979 -0.054734  0.056912   
              3050.0  3028.497972  195.268066  0.699723  0.082373  0.131852   
              3100.0  3028.497972  172.750244  0.702966  0.219100  0.205648   
              3150.0  3028.497972  152.212769  0.698962  0.351252  0.276081   
              4000.0  3028.497972   10.200851  0.191286  0.641202  0.368132   

                      strike_greek     delta     gamma  
contract bump strike                                    
N23      -2.0 3000.0     -0.001332  0.424206  0.001368  
              3050.0     -0.001250  0.359535  0.001306  
              3100.0     -0.001143  0.299612  0.001214  
              3150.0     -0.001019  0.245512  0.001099  
              4000.0     -0.000008  0.000890  0.000011  
...                            ...       ...       ...  
U23       2.0 3000.0     -0.000822  0.579489  0.000814  
              3050.0     -0.000821  0.538382  0.000827  
              3100.0     -0.000811  0.497537  0.000831  
              3150.0     -0.000794  0.457368  0.000826  
              4000.0     -0.000171  0.053327  0.000226  

[90 rows x 8 columns]

In [363]:
dfs.groupby(['contract','bump','strike']).mean()

S_0       price       rho      vega     theta  \
contract bump strike                                                          
N23      -2.0 3000.0  2919.230593   84.047241  0.426816  0.377628  0.669942   
              3050.0  2919.230593   66.389832  0.407454  0.570606  0.934510   
              3100.0  2919.230593   51.719666  0.378622  0.722290  1.138008   
              3150.0  2919.230593   39.737976  0.342915  0.825342  1.270979   
              4000.0  2919.230593    0.068120  0.003295  0.032490  0.046691   
...                           ...         ...       ...       ...       ...   
U23       2.0 3000.0  3028.497972  219.822144  0.688979 -0.054734  0.056912   
              3050.0  3028.497972  195.268066  0.699723  0.082373  0.131852   
              3100.0  3028.497972  172.750244  0.702966  0.219100  0.205648   
              3150.0  3028.497972  152.212769  0.698962  0.351252  0.276081   
              4000.0  3028.497972   10.200851  0.191286  0.641202  0.368132   

                      strike_greek     delta     gamma  
contract bump strike                                    
N23      -2.0 3000.0     -0.001332  0.424206  0.001368  
              3050.0     -0.001250  0.359535  0.001306  
              3100.0     -0.001143  0.299612  0.001214  
              3150.0     -0.001019  0.245512  0.001099  
              4000.0     -0.000008  0.000890  0.000011  
...                            ...       ...       ...  
U23       2.0 3000.0     -0.000822  0.579489  0.000814  
              3050.0     -0.000821  0.538382  0.000827  
              3100.0     -0.000811  0.497537  0.000831  
              3150.0     -0.000794  0.457368  0.000826  
              4000.0     -0.000171  0.053327  0.000226  

[90 rows x 8 columns]

In [792]:
l

,contract,-2.0,-1.5,-1.0,-0.5,0.0,0.5,1.0,1.5,2.0
0,K23,-0.025327,-0.018995,-0.012664,-0.006332,0.0,0.006332,0.012664,0.018995,0.025327
1,N23,-0.021378,-0.016033,-0.010689,-0.005344,0.0,0.005344,0.010689,0.016033,0.021378
2,U23,-0.019353,-0.014515,-0.009677,-0.004838,0.0,0.004838,0.009677,0.014515,0.019353


In [739]:
l

,contract,-2.0,-1.5,-1.0,-0.5,0.0,0.5,1.0,1.5,2.0
0,K23,-2.532732,-1.899549,-1.266366,-0.633183,0.0,0.633183,1.266366,1.899549,2.532732
1,N23,-2.137761,-1.603321,-1.068880,-0.534440,0.0,0.534440,1.068880,1.603321,2.137761
2,U23,-1.935307,-1.451480,-0.967654,-0.483827,0.0,0.483827,0.967654,1.451480,1.935307


In [737]:
l.assign(**l._get_numeric_data()*100)

TypeError: keywords must be strings

In [646]:

test = test.to_frame()

for i,p in enumerate(test):
    print(i,p)

0 0


In [653]:
pd.DataFrame(test, columns=['sdev'])

,sdev
K23,0.012664
N23,0.010689
U23,0.009677


In [60]:
    def shock_map(returns, window,bumps = [-2,-1.5,-1,-0.5,0,0.5,1,1.5,2]):
        shock_map = returns.copy()
        shock_map = shock_map[-window:].std()
        # shock_map = shock_map.to_frame()
        # shock_map = shock_map.to_frame().T
        shock_map*=bumps
        
        return shock_map

In [521]:
MONTHS = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
INT_TO_MONTHS = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
                 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
MONTHS_TO_INT = {v: k for k, v in INT_TO_MONTHS.items()}
MONTH_MAP = {'F': 1, 'G': 2, 'H': 3, 'J': 4, 'K': 5, 'M': 6, 'N': 7, 'Q': 8, 'U': 9, 'V': 10, 'X': 11, 'Z': 12}
YEAR_DAYS = 365
YEAR_BDAYS = 252

SEASONS = {'H1': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun'],
           'H2': ['Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
           'SumUS': ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'],
           'WinUS': ['Nov', 'Dec', 'Jan', 'Feb', 'Mar'],
           'Sum': ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep'],
           'Win': ['Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar'],
           }
SEASONS_COUPLES = {'season': ('H1', 'H2'),
                   'season_eu': ('Sum', 'Win'),
                   'season_us': ('SumUS', 'WinUS')
                   }
SEASONS_START = {'H1': 1,  'H2': 7, 'SumUS': 4, 'WinUS': 11, 'Sum': 4, 'Win': 10}
SEASONS_END = {'H1': 6,  'H2': 12, 'SumUS': 10, 'WinUS': 3, 'Sum': 9, 'Win': 3}






def _extract_monthly_spec(absolute: list, period_year: int):
    specs = [INT_TO_MONTHS[x - 12 * ((x - 1) // 12)] for x in absolute]
    year_shift = [x // 12 for x in absolute]
    years = [int(period_year + x) for x in year_shift]
    return years, specs


In [523]:
_extract_monthly_spec([1, 2, 3, 4, 5, 24], 2021)

([2021, 2021, 2021, 2021, 2021, 2023],
 ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Dec'])